# Event log 
Notebook for reading data from eventLog

In [2]:
import numpy as np
import pandas as pd
import h5py
import ast

eventArg  = {}
eventName = {}
eventlog  = {}
event = {}

path = '../datasets/'
key = '/events/'

fname = [
    'VRAcuityExp_2017-07-13_14-39-17_VR-4A_NIC',
    'VRAcuityExp_2017-07-13_15-05-16_VR-2B_NIC',
    'VRAcuityExp_2017-07-13_15-19-09_VR-2A_EDU',
    'VRAcuityExp_2017-07-13_15-38-34_VR-1A_NIC',
    'VRAcuityExp_2017-07-13_15-53-40_VR-1B_NIC',
    'VRAcuityExp_2017-07-13_16-11-46_VR-3A_NIC',
    'VRAcuityExp_2017-07-13_16-27-08_VR-3A_NIC',
    'VRAcuityExp_2017-07-13_17-09-07_VR-5A_NIC', ]


for i, x in enumerate(fname):
    eventlog[i]  = pd.read_hdf(path+x+'.h5', key+'eventlog')      

    file    = h5py.File(path+x+'.h5', 'r')
    eventArg[i]  = file[key]['eventArguments']
    eventName[i] = file[key]['eventNames']


In [214]:
def make_dict(event):
    eventD = {}
    for i in range(0, len(event)-1):
        # change to string in proper format
        str_dict = event[i].decode('UTF-8').replace('\'', '\"')

        # find location of "attr"
        n = str_dict.find('"attr"')

        if n != -1:
            str_dict = str_dict='{' + str_dict[n:]
            dictionary = ast.literal_eval(str_dict) 
            dictionary [dictionary['attr'] ] = dictionary.pop('value')
            del dictionary['attr']
        else:
            dictionary = ast.literal_eval(str_dict)
        eventD[i] = dictionary
    return eventD


# cleaning event arguments values
eventA = {}
for i, str_dict in enumerate(fname):
    eventA[i] = make_dict(eventArg[i])


In [225]:
# split into 3 dictionaries 
eventVis = {}
eventSpe = {}
eventDur = {}
    
j, m, n = 0, 0, 0
x = eventA[1]
for i in range(0,len(x)-1):
    k = list(x[i].keys())[0]
    if   k == 'visible':
        eventVis[m] = {'visible':x[i]['visible'], 'i':i}
        m += 1
    elif k == 'speed':
        eventSpe[n] = {'speed':x[i]['speed'], 'i':i}
        n += 1
    elif k == 'duration':
        eventDur[j] = {'duration':x[i]['duration'], 'i':i}
        j += 1
        
DFVis = pd.DataFrame.from_dict(eventVis, orient='index')
DFSpe = pd.DataFrame.from_dict(eventSpe, orient='index')
DFDur = pd.DataFrame.from_dict(eventDur, orient='index')
#print(DFVis)
#print(DFSpe)
#print(DFDur)

In [223]:
# create new df for merge
DFV = pd.DataFrame()

l = DFVis.i[len(DFVis)-1]
j = 0
for i in range(0, l+1):
    if i == DFVis.i[j]:
        DFV[i] = DFVis.iloc[j]
        j += 1
    else:
        DFV[i] = DFVis.iloc[j]

DFV = DFV.transpose()
#print(DFV)

In [230]:
# create new df for merge
def index_df(df):
    dfOut = pd.DataFrame()
    l = df.i[len(df)-1]
    j = 0
    for i in range(0, l+1):
        if i == df.i[j]:
            dfOut[i] = df.iloc[j]
            j += 1
        else:
            dfOut[i] = df.iloc[j]

    return dfOut.transpose()

DFV = index_df(DFVis)
#print(DFSpe)
#print(DFDur)

print(DFV)

   visible   i
0    False   0
1     True   2
2     True   2
3    False   5
4    False   5
5    False   5
6     True   7
7     True   7
8    False  10
9    False  10
10   False  10
11    True  12
12    True  12
13   False  15
14   False  15
15   False  15
16    True  17
17    True  17
18   False  20
19   False  20
20   False  20
21    True  22
22    True  22
23   False  25
24   False  25
25   False  25
26    True  27
27    True  27
28   False  30
29   False  30
..     ...  ..
51    True  52
52    True  52
53   False  55
54   False  55
55   False  55
56    True  57
57    True  57
58   False  60
59   False  60
60   False  60
61    True  62
62    True  62
63   False  65
64   False  65
65   False  65
66    True  67
67    True  67
68   False  70
69   False  70
70   False  70
71    True  72
72    True  72
73   False  75
74   False  75
75   False  75
76    True  77
77    True  77
78   False  80
79   False  80
80   False  80

[81 rows x 2 columns]


In [210]:
#print(eventlog[1])